In [1]:
import json

import chess
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [2]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

In [3]:
# Global variables:

USED_BOARDS = set()
SWAPS = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
        "diagonal" : {x : ((x >> 3) | (x << 3)) & 63 for x in range(64)},
        "anti_diagonal" : {x : (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
    }
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("P")]
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
FILENAME = "test_15_05"

# Überlegung im Vergleich zu vorherigen Version:
* Muss der Nutzer wirklich einen Namen der Datei angeben? Können wir diesen nicht generisch erstellen?

# Voraussetzungen:
* Nutzer ist immer weiß

# Strukturüberlegung:
* Zentrale used_boards -> Müssen nicht übergeben werden
* Uniques werden gespeichert anstatt kompletter s_n Menge
* Angabe des Nutzers der aktuellen Situation
* Nutzer startet Berechnung:

# Benötigte Funktionen:

* Spiegelungen aus Uniques generieren; gen_mirroring(uniques):
    * Über Uniques iterieren
        * Über Spiegelarten iterieren
            * Spiegelung durchführen (ohne Objekterstellung in Integer-Schreibweise spiegeln)
            * Spiegelungen in Menge zusammenfassen
    * Menge zurückgeben

Menge an Schachbrettern in alle Richtungen spiegeln (falls überhaupt benötigt)

In [4]:
def gen_mirroring(uniques):
    result = uniques.copy()
    for unique in uniques:
        result |= mirror_board_all(unique)
    return result

Schachbrett in alle Richtungen spiegeln

In [5]:
# Implementierung ohne ENUM:
def mirror_board_all(board_int):
    result = set()
    for name, swap in SWAPS.items():
        result.add(mirror_board_7(board_int, swap))
    return result

Mirror Board with bits in one direction

In [6]:
# Mirroring mit 6 Bit Darstellung

def mirror_board(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in list -> Überlegung Liste global zu machen
    rounds = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1
    for index in range(0, rounds):
        result = (mirror[board_int & 63] << (6 * index) + 1) | result
        board_int = board_int >> 6
    return result

In [7]:
# TODO: Erstellen nach Lukas seiner Bitmethode, wenn ich sie komplett
# Mirroring mit 7 Bit Darstellung
def mirror_board_7(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1
    
    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result

* Start der Berechnung; start_calculation():
    * Bekommt Positionen der Figuren übergeben
    * Berechnung der ersten Menge S_0, damit auch Berechnung der ersten Uniques -> Rückgabe nur Uniques
        * Hierbei erste wichtige Frage: Werden in used_boards dann ebenfalls nur die Uniques gespeichert oder auch alle?
        * Schreiben wir die ganze S_0 Menge direkt in die Datei nach der Berechnung?
    * Schleife iterieren mit altbekannter Bedingung (wenn neue Menge leer oder sich nicht unterscheidet)
        * Berechnung nächster Menge mit Übergabe der Uniques vorheriger Menge

In [8]:
def calculate():
    #sum = 0
    sn = calculate_s0()
    # Wirklich nur für eine Ausgabe spiegeln?
    #len_mir = len(gen_mirroring(sn))
    #sum += len_mir
    #print("Done with S" + str(0) + " - length: " + str(len(sn)) + " - actual length: " + str(len_mir))
    print("Done with S" + str(0))
    store_sn(sn)
    
    count = 0
    
    while len(sn) != 0:
        sn = calculate_next_sn(sn)
        store_sn(sn)
        count += 1
        #len_mir = len(gen_mirroring(sn))
        #sum += len_mir
        #print("Done with S" + str(count) + " - length: " + str(len(sn)) + " - actual length: " + str(len_mir))
        print("Done with S" + str(count))
    print("Done")
    #print("Total: " + str(sum))

## Erstellung von S_0
> TODO: Einleitung schreiben

> TODO: Beschreibung von calculate_s0() schreiben

In [9]:
def calculate_s0():
    # TODO: Hier muss noch irgendwo eine Berücksichtigug rein, dass der Bauer durch die Queen beim Erstellen von s0  ersetzt wird, dies muss dazu noch rückgängi gemacht werden, wenn die Queen durch den Pawn ersetzt wird.
    global PIECE_LIST

    s0 = set()
    empty_board = chess.Board(None)
    # Turn has to be black while checkmate
    empty_board.turn = chess.BLACK

    # Create temp_piece_list for conversion between board_object and integer
    tmp_piece_list = []
    s0.add(to_integer(empty_board, tmp_piece_list))
   
    if USER_WANTS_PAWN: 
        PIECE_LIST[PIECE_LIST.index(chess.Piece.from_symbol('P'))] = chess.Piece.from_symbol('Q')

    for piece in PIECE_LIST:
        s0 = fill_boards_with_piece(s0, piece, tmp_piece_list)
        tmp_piece_list.append(piece)
        
    return s0

> TODO: Beschreibung von fill_boards_with_piece() schreiben

In [10]:
def fill_boards_with_piece(int_boards_set, piece, cur_piece_list):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(PIECE_LIST)
    for int_board in int_boards_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    # Don't process invalid boards further
                    # than the second king
                    continue

                if board_object.is_checkmate():
                    if board_int not in USED_BOARDS:
                        finished_boards.add(board_int)
                        add_to_used(board_int)
                        continue

                if len(used_squares) + 1 < piece_count: #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
    return finished_boards

> TODO: Beschreibung von add_board_with_reflection() schreiben.
> TODO: Schauen, ob sie auch für die Rückwärtsgenerierung verwendet werden kann.

In [11]:
def add_all_to_used(boards):
    for board in boards:
        add_to_used(board)

In [12]:
def add_to_used(board_int):
    global USED_BOARDS
    USED_BOARDS.add(board_int)
    if not USER_WANTS_PAWN:
        for reflection in mirror_board_all(board_int):
            USED_BOARDS.add(reflection)

> TODO: Beschreibung von store_sn schreiben

> Momentaner start hier positioniert, um zu zeigen, dass s0 funktioniert
> Bemerkung: Es müssen alle Zellen davor ausgeführt werden, dass es funktioniert! (Genauer: Es muss USED_BOARDS resetted werden!)

In [13]:
def calculate_next_sn(sn):
    sn_p_1 = set()
    for board_int in sn:
        # Nicht sicher, ob hier schon die Bretterstellung notwendig ist (glaube nicht)
        chess_board = to_board(board_int, PIECE_LIST)
        chess_board.turn = chess_board.turn ^ True

        # Berücksichtigung bei dieser Trennung, dass irgendwann auch der Pawn in eine Queen umgewandelt wird und dann muss USER_WANTS_PAWN = False sein -> Danach aber vorsichtig bei Spiegelungen sein, weil diese auch nicht instant verwendet werden können -> Möglicherweise doch so umsetzbar
        
        if need_pawn_moves(chess_board):
            reached_boards = do_pawn_stuff(chess_board)
        else:
            reached_boards = do_not_pawn_stuff(chess_board)
            
        if not chess_board.turn:
            reached_boards = check_black_determinism(reached_boards)
        
        reached_integers = multiple_to_integer(reached_boards)
        
        # Remove unwanted mirrors
        reached_integers = reached_integers - USED_BOARDS
        
        sn_p_1 |= reached_integers
        add_all_to_used(reached_integers)

    return sn_p_1

> TODO: Diese Funktion wird verwendet, um jeglichen Stuff bzgl. Pawns abzuarbeiten -> Modularisierung von Funktionen und Trennung der Funktionalität

# Beriech für die Implementierung von nicht-Bauern stuff

In [14]:
def do_not_pawn_stuff(chess_board):
    tmp = []
    for pLMove in chess_board.pseudo_legal_moves:
        # Backwards-execuion of Move
        chess_board.push(pLMove)
        chess_board.turn = chess_board.turn ^ True
        tmp.append(chess_board.copy())
        chess_board.turn = chess_board.turn ^ True
        chess_board.pop()
        
    reached_boards = keep_if_valid(tmp)  

    return reached_boards

In [15]:
'''
# Check if Move is valid -> Kann man Auslagern? Unsicher wegen continue? Maximal, dass man in der Funktion ein Bool weitergibt und anhand davon etwas macht

        #why outcome?
        if not chess_board.is_valid() or chess_board.outcome() is not None:
            chess_board.turn = chess_board.turn ^ True
            chess_board.pop()
            continue

        new_board_int = to_integer(chess_board, PIECE_LIST)

        # Erstellung der Mirrored Boards und das Hinzufügen in Uniques passiert erst in check_black_determinism bzw. mal schauen, wo es genau passiert
        if new_board_int not in USED_BOARDS:
            sn_p_1.add(new_board_int)
'''

'\n# Check if Move is valid -> Kann man Auslagern? Unsicher wegen continue? Maximal, dass man in der Funktion ein Bool weitergibt und anhand davon etwas macht\n\n        #why outcome?\n        if not chess_board.is_valid() or chess_board.outcome() is not None:\n            chess_board.turn = chess_board.turn ^ True\n            chess_board.pop()\n            continue\n\n        new_board_int = to_integer(chess_board, PIECE_LIST)\n\n        # Erstellung der Mirrored Boards und das Hinzufügen in Uniques passiert erst in check_black_determinism bzw. mal schauen, wo es genau passiert\n        if new_board_int not in USED_BOARDS:\n            sn_p_1.add(new_board_int)\n'

# Bereich für alle Bauern-Funktionen

In [16]:
def do_pawn_stuff(chess_board):
    # Frage: Brauche ich pawn_positions überhaupt, wenn ich das so trenne, in alter Version werden sie nur zur Überprüfung verwendet, ob Pawns auf dem Feld sind (wissen wir durch USER_WANTS_PAWN) und ob pseudo_legal_moves Pawns enthalten (da wir dieses Szenario woanders behandeln (ohne Pawns) ist das irrelevant;

    # Bezüglich der Trennung mit dem anderen shit, wenn eine Queen auf dem Brett steht, muss hier eine Funktion definiert werden, die die Züge der Queen rückwärts darstellt! Glaube die Position von do_queen_backwards sollte passen()
    
    tmp = []

    # Es stehe Pawns auf dem Schachbrett
    if check_for_pawn(chess_board):
        tmp = pawn_moves(chess_board)
    # Stehen keine Pawns, also irgendwelche anderen Figuren, die mal ein Pawn waren, auf dem Schachbrett, da Queen optimal, Queens
    else:
        tmp = [replace_queen(chess_board)]
    
    reached_boards = keep_if_valid(tmp)

    return reached_boards

In [17]:
def check_for_pawn(chess_board):
    return len(set(chess_board.pieces(chess.PAWN, chess.WHITE))) > 0

In [18]:
def need_pawn_moves(chess_board):
    return USER_WANTS_PAWN and chess_board.turn and (check_for_pawn(chess_board) or check_top_row_for_queen(chess_board))

In [19]:
def find_pawn(chess_board):
    return list(chess_board.pieces(chess.PAWN, chess.WHITE))[0]

In [20]:
def pawn_moves(chess_board):
    # Müssen auch Anfangszüge berücksichtigt werden? Also Doppelzug? Wenn ja, dann pawn_moves, wenn nein, dann pawn_move
    
    res = []
    position = find_pawn(chess_board)
    if position in range(24-31):
        tmp_board = chess_board.copy()
        tmp_board.remove_piece_at(position)
        tmp_board.set_piece_at(position - 16, chess.Piece.from_symbol('P'))
        res.append(tmp_board)
    
    chess_board.remove_piece_at(position)
    chess_board.set_piece_at(position - 8, chess.Piece.from_symbol('P'))
    res.append(chess_board)
    
    return res

In [21]:
def check_top_row_for_queen(chess_board):
    squares_with_queen = set(chess_board.pieces(chess.QUEEN, chess.WHITE))
    for i in range(56,64):
        if i in squares_with_queen:
            return True
    return False

In [22]:
def replace_queen(chess_board):
    # Same case as find_pawn, es kann doch nur eine toprow_queen_position geben
    queen_square = list(chess_board.pieces(chess.QUEEN, chess.WHITE))[0]
    chess_board.remove_piece_at(queen_square)
    chess_board.set_piece_at(queen_square - 8, chess.Piece.from_symbol('P'))
    
    return chess_board

> TODO: Funktion, die zur Generierung von anderen Zügen ohne Bauern benötigt wird.

* Funktion zur Überprüfung der Züge von schwarz (dass diese auch bei einer Menge m landen mit m < n)
    * Einfach überprüfen, ob Züge in used_boards landen

## Operationen für die Rechnung

In [23]:
def keep_if_valid(boards):
    res = []
    for board in boards:
        if board.is_valid():
            res.append(board)
            
    return res

In [24]:
def check_black_determinism(boards):
    deterministic = []
    for chess_board in boards:
        include = True
        for move in chess_board.legal_moves:
            chess_board.push(move)
            rep = to_integer(chess_board,PIECE_LIST)
            chess_board.pop()
            if rep not in USED_BOARDS:
                include = False
                break
        
        if include:
            deterministic.append(chess_board)
            
    return deterministic

In [25]:
def multiple_to_integer(boards):
    integers = set()
    for board in boards:
        integers.add(to_integer(board, PIECE_LIST))
    return integers


## Storage Stuff

In [29]:
def store_sn(sn):
    f = open("S_n_Results/" + FILENAME + ".preConvert", "a")
    f.write("\n")
    f.write(json.dumps(list(sn)))
    f.close()

In [30]:
def convert_temp_file():
    f = open("S_n_Results/" + FILENAME + ".preConvert", "rb")
    lines = f.readlines()
    
    data = [PIECE_LIST]
    
    for line in lines[1:]:
        list = json.loads(line)
        data.append(set(list))
        
    f = open("S_n_Results/" + FILENAME + ".pickle", "wb")
    f.write(pickle.dumps(data))
    f.close()
    
    with ZipFile("S_n_Results/" + FILENAME + '.chessAI', 'w', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + FILENAME + ".pickle", FILENAME + ".pickle")
    if os.path.exists("S_n_Results/" + FILENAME + ".chessAI") and os.path.exists("S_n_Results/" + FILENAME + ".pickle"):
        os.remove("S_n_Results/" + FILENAME + ".pickle") 

    if os.path.exists("S_n_Results/" + FILENAME + ".preConvert"):
        os.remove("S_n_Results/" + FILENAME + ".preConvert")

In [31]:
def run():
    create_empty_file()
    calculate()
    convert_temp_file()


In [32]:
%%time
# Scenario 1
USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("Q")]
FILENAME = "s_n_queen_24_05"
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
run()

Done with S0
Done with S1
Done with S2
Done with S3
Done with S4
Done with S5
Done with S6
Done with S7
Done with S8
Done with S9
Done with S10
Done with S11
Done with S12
Done with S13
Done with S14
Done with S15
Done with S16
Done with S17
Done with S18
Done with S19
Done with S20
Done with S21
Done
CPU times: total: 1min 30s
Wall time: 1min 34s


In [33]:
%%time
# Scenario 2
USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("R")]
FILENAME = "s_n_rook_24_05"
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
run()

Done with S0
Done with S1
Done with S2
Done with S3
Done with S4
Done with S5
Done with S6
Done with S7
Done with S8
Done with S9
Done with S10
Done with S11
Done with S12
Done with S13
Done with S14
Done with S15
Done with S16
Done with S17
Done with S18
Done with S19
Done with S20
Done with S21
Done with S22
Done with S23
Done with S24
Done with S25
Done with S26
Done with S27
Done with S28
Done with S29
Done with S30
Done with S31
Done with S32
Done with S33
Done
CPU times: total: 58.5 s
Wall time: 1min


In [34]:
%%time
# Scenario 3
USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("B"), chess.Piece.from_symbol("B")]
FILENAME = "s_n_two_bishop_24_05"
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
run()

Done with S0
Done with S1
Done with S2
Done with S3
Done with S4
Done with S5
Done with S6
Done with S7
Done with S8
Done with S9
Done with S10
Done with S11
Done with S12
Done with S13
Done with S14
Done with S15
Done with S16
Done with S17
Done with S18
Done with S19
Done with S20
Done with S21
Done with S22
Done with S23
Done with S24
Done with S25
Done with S26
Done with S27
Done with S28
Done with S29
Done with S30
Done with S31
Done with S32
Done with S33
Done with S34
Done with S35
Done with S36
Done with S37
Done with S38
Done with S39
Done
CPU times: total: 34min 57s
Wall time: 36min 35s


In [35]:
%%time
# Scenario 4
USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("B"), chess.Piece.from_symbol("N")]
FILENAME = "s_n_knight_bishop_24_05"
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
run()

Done with S0
Done with S1
Done with S2
Done with S3
Done with S4
Done with S5
Done with S6
Done with S7
Done with S8
Done with S9
Done with S10
Done with S11
Done with S12
Done with S13
Done with S14
Done with S15
Done with S16
Done with S17
Done with S18
Done with S19
Done with S20
Done with S21
Done with S22
Done with S23
Done with S24
Done with S25
Done with S26
Done with S27
Done with S28
Done with S29
Done with S30
Done with S31
Done with S32
Done with S33
Done with S34
Done with S35
Done with S36
Done with S37
Done with S38
Done with S39
Done with S40
Done with S41
Done with S42
Done with S43
Done with S44
Done with S45
Done with S46
Done with S47
Done with S48
Done with S49
Done with S50
Done with S51
Done with S52
Done with S53
Done with S54
Done with S55
Done with S56
Done with S57
Done with S58
Done with S59
Done with S60
Done with S61
Done with S62
Done with S63
Done with S64
Done with S65
Done
CPU times: total: 57min 29s
Wall time: 59min 54s


In [32]:
%%time
# Scenario 5
USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("P")]
FILENAME = "s_n_pawn_24_05"
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
run()

Done with S0
Done with S1
Done with S2
Done with S3
Done with S4
Done with S5
Done with S6
Done with S7
Done with S8
Done with S9
Done with S10
Done with S11
Done with S12
Done with S13
Done with S14
Done with S15
Done with S16
Done with S17
Done with S18
Done with S19
Done with S20
Done with S21
Done with S22
Done
CPU times: total: 6min 34s
Wall time: 6min 56s
